In [2]:
import pandas as pd

In [4]:
from pip._internal import main as pipmain

from cifo.algorithm.genetic_algorithm import GeneticAlgorithm
from cifo.algorithm.hill_climbing import HillClimbing
from cifo.custom_problem.knapsack_problem import (
    KnapsackProblem, knapsack_decision_variables_example, knapsack_constraints_example, 
    knapsack_bitflip_get_neighbors
)
from cifo.custom_problem.travel_salesman_problem import (
    TravelSalesmanProblem, tsp_bitflip_get_neighbors
)

from cifo.custom_problem.portfolio_investment_problem import (
    PortfolioInvestmentProblem, pip_bitflip_get_neighbors
)

from cifo.problem.objective import ProblemObjective
from cifo.algorithm.ga_operators import (
    initialize_randomly,
    RouletteWheelSelection, RankSelection, TournamentSelection, 
    singlepoint_crossover,
    single_point_mutation,
    elitism_replacement, standard_replacement 
)    
from cifo.util.terminal import Terminal, FontColor
from cifo.util.observer import GeneticAlgorithmObserver
from random import randint

In [5]:
from copy import deepcopy
from random import choice, randint

from cifo.problem.problem_template import ProblemTemplate
from cifo.problem.objective import ProblemObjective
from cifo.problem.solution import LinearSolution, Encoding

In [44]:
#read PIP datafiles into pandas df
df_PIP = pd.read_excel(r'./data/sp500_gen.xlsx')
#df_PIP.set_index('symbol', inplace=True)
df_PIP.rename(columns={'symbol':'stock', 'name': 'stock_name', 'exp_return_3m': 'exp_ret', 'standard_deviation': 'stdiv'}, inplace=True)

In [49]:
pip_dv = df_PIP.to_dict('list')

In [50]:
df_PIP.head(5)

,stock,stock_name,price,exp_ret,stdiv
0,A,Agilent Technologies,78.52,10.40,3.887141
1,AAL,American Airlines Gp,28.23,9.29,8.730168
2,AAP,Advance Auto Parts Inc,161.74,19.47,15.315660
3,AAPL,Apple Inc,263.19,25.11,8.398204
4,ABBV,Abbvie Inc,87.20,31.31,28.781523


In [51]:
for key in pip_dv:
    print(key)

stock
stock_name
price
exp_ret
stdiv


In [54]:
decision_variables = pip_dv

In [55]:
pip_problem_instance = PortfolioInvestmentProblem (
    decision_variables = df_PIP,
    constraints = {"Max-Investment" : 100000, "Min-Sharp-Ratio": 1}
)

AttributeError: 'PortfolioInvestmentProblem' object has no attribute '_values'

In [57]:
stocks = []
        #if decision_variables['symbol']:

if "stock" in decision_variables:
    stocks = decision_variables["stock"]

In [88]:
df_PIP.head()

,stock,stock_name,price,exp_ret,stdiv
0,A,Agilent Technologies,78.52,10.40,3.887141
1,AAL,American Airlines Gp,28.23,9.29,8.730168
2,AAP,Advance Auto Parts Inc,161.74,19.47,15.315660
3,AAPL,Apple Inc,263.19,25.11,8.398204
4,ABBV,Abbvie Inc,87.20,31.31,28.781523


In [73]:
current_pfolio = ['AOS', 'ATO', 'ATVI']

In [74]:
for stock in current_pfolio:
    i = pip_dv['stock'].index(stock)
    print(pip_dv['price'][i])

48.09
108.79
54.08


In [75]:
def cal_total_investment(porfolio):
    total_investment = 0
    for stock in porfolio:
        i = pip_dv['stock'].index(stock)
        total_investment += float(pip_dv['price'][i])
    return total_investment 

In [76]:
cal_total_investment(current_pfolio)

210.95999999999998

In [84]:
def cal_total_std(porfolio):
    list_std = []
    for stock in porfolio:
        i = pip_dv['stock'].index(stock)
        list_std.append(float(pip_dv['stdiv'][i]))
    return list_std

In [85]:
total_std = cal_total_std(current_pfolio)

print(total_std)

[1.9083712289609407, 1.0908819680992559, 3.2183422479764383]


In [86]:
print(numpy.corrcoef((total_std)))

1.0


In [95]:
import numpy as np

In [68]:
print(numpy.corrcoef((pip_dv['stdiv'])))

1.0


In [101]:
StockReturns = pd.Series()

In [91]:
StockReturns = df_PIP['exp_ret']

In [102]:
portfolio_weights = np.array([0.25, 0.35, 0.10, 0.20, 0.10])
port_ret = StockReturns.mul(portfolio_weights, axis=1).sum(axis=1)
port_ret

ValueError: No axis named 1 for object type <class 'pandas.core.series.Series'>

In [104]:
df_PIP.head()

,stock,stock_name,price,exp_ret,stdiv
0,A,Agilent Technologies,78.52,10.40,3.887141
1,AAL,American Airlines Gp,28.23,9.29,8.730168
2,AAP,Advance Auto Parts Inc,161.74,19.47,15.315660
3,AAPL,Apple Inc,263.19,25.11,8.398204
4,ABBV,Abbvie Inc,87.20,31.31,28.781523


In [145]:
temp_df = df_PIP.T.copy()

In [146]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,493,494,495,496,497,498,499,500,501,502
stock,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
stock_name,Agilent Technologies,American Airlines Gp,Advance Auto Parts Inc,Apple Inc,Abbvie Inc,Amerisourcebergen Corp,Abiomed Inc,Abbott Laboratories,Accenture Plc,Adobe Systems Inc,...,XCEL Energy Inc,Xilinx Inc,Exxon Mobil Corp,Dentsply Sirona Inc,Xerox Corp,Xylem Inc,Yum! Brands,Zimmer Biomet Holdings,Zions Bancorp,Zoetis Inc Cl A
price,78.52,28.23,161.74,263.19,87.2,87.55,185.25,83.89,197.71,300.1,...,61.69,91.01,68.03,56.63,38.3,77.47,98.12,142.57,48.88,119.56
exp_ret,10.4,9.29,19.47,25.11,31.31,1.45,-3.83,-0.77,1.81,5.04,...,-1.14,-13.41,-1.45,6.59,36.4,1.88,-15.83,3.35,20.76,-5.15
stdiv,3.88714,8.73017,15.3157,8.3982,28.7815,1.53219,2.86873,1.85436,1.63156,3.33591,...,1.94181,11.3276,1.95391,4.18521,19.9459,1.65691,7.91774,2.06521,17.3792,2.4498


In [147]:
temp_df.corr()

""


In [148]:
column_names = temp_df.iloc[0]
column_names

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
498     XYL
499     YUM
500     ZBH
501    ZION
502     ZTS
Name: stock, Length: 503, dtype: object

In [149]:
temp_df.columns = column_names

In [179]:
temp_df=temp_df.drop(temp_df.index[[0,1]])

In [156]:
new = temp_df.cov()

In [180]:
temp_df.head()

stock,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
price,78.52,28.23,161.74,263.19,87.2,87.55,185.25,83.89,197.71,300.1,...,61.69,91.01,68.03,56.63,38.3,77.47,98.12,142.57,48.88,119.56
exp_ret,10.4,9.29,19.47,25.11,31.31,1.45,-3.83,-0.77,1.81,5.04,...,-1.14,-13.41,-1.45,6.59,36.4,1.88,-15.83,3.35,20.76,-5.15
stdiv,3.88714,8.73017,15.3157,8.3982,28.7815,1.53219,2.86873,1.85436,1.63156,3.33591,...,1.94181,11.3276,1.95391,4.18521,19.9459,1.65691,7.91774,2.06521,17.3792,2.4498


In [181]:
temp_df.index

Index(['price', 'exp_ret', 'stdiv'], dtype='object')

In [182]:
temp_df = temp_df.apply(pd.to_numeric)


In [183]:
cov_mat = temp_df.cov()

stock,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
stock,,,,,,,,,,,,,,,,,,,,,
A,1708.802567,458.194525,3441.092699,5890.448566,1363.938728,2047.391977,4408.022893,1978.750292,4663.294141,7043.156364,...,1453.181095,2149.817119,1607.042645,1223.081873,267.735170,1801.465244,2389.943447,3329.718106,714.757924,2864.336277
AAL,458.194525,123.213412,925.361604,1581.164719,366.520438,551.336813,1188.971538,533.612082,1255.657105,1896.040067,...,392.225770,586.276534,433.753041,328.628072,67.183269,485.023675,650.641191,896.227669,191.457621,773.563669
AAP,3441.092699,925.361604,6949.683143,11874.798119,2752.643563,4140.701123,8929.606657,4007.615114,9430.348449,14239.791929,...,2945.768455,4403.417814,3257.654395,2468.062406,504.359573,3642.666310,4886.802724,6730.914789,1437.860116,5809.783833
AAPL,5890.448566,1581.164719,11874.798119,20313.371638,4705.515870,7068.991157,15228.911659,6835.667331,16100.338711,24314.903089,...,5021.735475,7458.220659,5553.426398,4219.368829,900.634402,6219.458157,8285.857296,11494.406300,2462.908875,9900.444685
ABBV,1363.938728,366.520438,2752.643563,4705.515870,1090.467288,1639.491778,3534.196603,1586.234851,3733.984343,5638.619706,...,1165.695793,1738.068234,1289.115830,977.760104,203.335666,1442.362409,1929.683063,2665.388411,570.067268,2298.704227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL,1801.465244,485.023675,3642.666310,6219.458157,1442.362409,2171.592789,4686.325086,2103.039931,4945.572237,7467.094725,...,1546.387871,2321.426266,1710.111059,1293.176912,256.462261,1910.253761,2574.449087,3529.338191,752.419625,3050.603249
YUM,2389.943447,650.641191,4886.802724,8285.857296,1929.683063,2928.674587,6359.324668,2851.521629,6667.547579,10058.401591,...,2103.681041,3278.942183,2326.382269,1729.264653,246.889866,2574.449087,3614.167506,4751.242950,994.238444,4159.131146
ZBH,3329.718106,896.227669,6730.914789,11494.406300,2665.388411,4012.108539,8656.759002,3884.899110,9137.231396,13796.203343,...,2856.356747,4283.554669,3158.773271,2389.733747,477.416438,3529.338191,4751.242950,6520.909403,1390.870491,5634.484387


In [185]:
temp_df.columns

Index(['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE',
       ...
       'XEL', 'XLNX', 'XOM', 'XRAY', 'XRX', 'XYL', 'YUM', 'ZBH', 'ZION',
       'ZTS'],
      dtype='object', name='stock', length=503)

In [202]:
lista = ['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'A']

In [205]:
list_set = list(set(lista))

In [206]:
list_set

['AAPL', 'A', 'AAL', 'ABBV', 'AAP']

In [207]:
for _ in list_set:
    print(1/len)

AAPL
A
AAL
ABBV
AAP


In [209]:
len(list_set)

5

In [226]:
from collections import Counter

names = ['adam','josh','drake']

count = Counter(names).items()


percentages = {x: float(float(y) / len(names) * 100.00) for x, y in count}

weights = np.array(percentages.values())
stocks = percentages.keys()
print(weights)
print(stocks)

dict_values([33.33333333333333, 33.33333333333333, 33.33333333333333])
dict_keys(['adam', 'josh', 'drake'])


In [190]:
df_PIP.columns

Index(['stock', 'stock_name', 'price', 'exp_ret', 'stdiv'], dtype='object')

In [195]:
tiny_df = df_PIP.loc[:20, ['stock','exp_ret']]

In [197]:
tiny_df.set_index('stock', inplace = True)

In [199]:
tiny_df = tiny_df.T

In [200]:
tiny_df

stock,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,ADM,ADP,ADS,ADSK,AEE,AEP,AES,AFL,AGN,AIG
exp_ret,10.4,9.29,19.47,25.11,31.31,1.45,-3.83,-0.77,1.81,5.04,...,13.39,1.61,-22.59,12.81,-1.55,0.76,22.45,4.38,16.25,-0.94


In [201]:
tiny_df.cov()

/home/kalrashid/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7639: RuntimeWarning: Degrees of freedom <= 0 for slice
  baseCov = np.cov(mat.T)
/home/kalrashid/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/home/kalrashid/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


stock,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,ADM,ADP,ADS,ADSK,AEE,AEP,AES,AFL,AGN,AIG
stock,,,,,,,,,,,,,,,,,,,,,
A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [235]:
df_cov = pd.read_excel(r'./data/sp_12_weeks.xlsx')

In [236]:
df_cov.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
0,78.520000,28.230000,161.740000,263.190000,87.200000,87.550000,185.250000,83.890000,197.710000,300.100000,...,61.690000,91.010000,68.030000,56.630000,38.300000,77.470000,98.120000,142.570000,48.880000,119.560000
1,80.206086,28.731618,158.045930,257.029252,89.171711,86.885944,183.994713,86.787228,201.930767,306.696578,...,62.239853,91.520147,68.986101,58.593867,38.291229,78.665575,97.149740,145.650724,48.548460,118.836005
2,78.384788,28.278183,164.420926,257.929716,85.567285,91.174655,189.248009,86.382276,192.433079,306.869328,...,61.360148,90.273761,67.313634,56.193379,38.068402,79.757736,101.315156,140.696998,48.753962,122.872459
3,81.511740,29.824899,166.672193,258.302726,86.839031,85.954691,192.069035,82.435135,195.138094,302.881966,...,61.570786,91.252331,66.089838,55.662228,39.797512,76.650129,101.998646,139.294932,50.570746,121.813726
4,79.857587,28.236047,166.233464,268.579908,86.092338,86.630503,188.019652,83.403046,203.054404,310.147192,...,64.242991,90.816297,68.294200,55.683695,39.438432,78.022546,99.679349,142.479663,49.251931,116.432194


In [237]:
cov_mat = df_cov.cov()

In [239]:
cor_mat = df_cov.corr()

In [240]:
cor_mat

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
A,1.000000,0.373153,0.487860,-0.024923,-0.004408,-0.145108,-0.053462,-0.316669,0.116548,0.239280,...,0.003771,-0.084702,0.226630,-0.006024,0.734769,-0.436787,0.309888,-0.188042,0.416300,-0.091145
AAL,0.373153,1.000000,0.302599,-0.008552,0.333694,-0.024345,-0.299035,-0.300660,0.437044,-0.090520,...,0.401533,-0.112476,0.043854,0.066094,0.144895,-0.045704,0.234357,-0.004914,0.409517,0.481010
AAP,0.487860,0.302599,1.000000,-0.056387,-0.334238,0.089623,0.053124,-0.379132,-0.027284,0.147197,...,0.077332,-0.069297,-0.019082,-0.543483,0.521296,-0.159424,0.501052,-0.240976,0.167176,0.169634
AAPL,-0.024923,-0.008552,-0.056387,1.000000,-0.154245,0.159582,0.039724,-0.200262,0.481990,0.176010,...,0.505686,-0.598870,0.381644,0.061084,0.223065,-0.230602,0.102973,-0.250258,0.606150,-0.052960
ABBV,-0.004408,0.333694,-0.334238,-0.154245,1.000000,-0.300683,-0.594541,-0.167478,0.579972,-0.296514,...,0.380543,-0.116643,0.161907,0.417328,-0.157853,0.089952,-0.059608,0.518148,-0.088265,0.329842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL,-0.436787,-0.045704,-0.159424,-0.230602,0.089952,0.319051,-0.354318,-0.036011,0.008312,0.341286,...,0.039441,0.324289,0.378489,-0.173092,-0.275700,1.000000,0.221015,0.351856,-0.309030,-0.059964
YUM,0.309888,0.234357,0.501052,0.102973,-0.059608,0.039083,0.102613,-0.697632,-0.132936,0.366206,...,0.112729,-0.247551,0.192965,-0.532262,0.645739,0.221015,1.000000,-0.404270,0.618691,0.200642
ZBH,-0.188042,-0.004914,-0.240976,-0.250258,0.518148,0.297128,-0.624465,0.154745,0.367769,-0.404867,...,-0.045989,0.481878,0.299354,0.486543,-0.342879,0.351856,-0.404270,1.000000,-0.596191,0.101672
ZION,0.416300,0.409517,0.167176,0.606150,-0.088265,-0.115967,0.257567,-0.570984,0.132039,0.242639,...,0.305282,-0.490717,0.138434,-0.175679,0.650290,-0.309030,0.618691,-0.596191,1.000000,0.094937


In [273]:
df_cov.columns

Index(['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE',
       ...
       'XEL', 'XLNX', 'XOM', 'XRAY', 'XRX', 'XYL', 'YUM', 'ZBH', 'ZION',
       'ZTS'],
      dtype='object', length=503)

In [242]:
stocks = ['A', 'AAL', 'AAP', 'ABC']

In [243]:
df_stocks_cov = df_cov[stocks]

In [246]:
cov_mat = df_stocks_cov.cov()

In [ ]:
        # calcuate covariance of stocks from sp_12_weeks
        #then pfolio_risk = np.sqrt(np.dot(weight_of_stocks.T, np.dot(cov_mat, weight_of_stocks)))

In [261]:
weights = [22,33,21,23]

In [267]:
weight_of_stocks = np.array(weights)

In [268]:
np.dot(cov_mat, weight_of_stocks)

array([ 71.42136605,  30.99424175, 238.04763458,  99.29943304])

In [269]:
pfolio_risk = np.sqrt(np.dot(weight_of_stocks.T, np.dot(cov_mat, weight_of_stocks)))

In [270]:
pfolio_risk

99.38293272430677

In [256]:
cov_mat

,A,AAL,AAP,ABC
A,1.543100,0.297484,1.723450,-0.371142
AAL,0.297484,0.411870,0.552274,-0.032169
AAP,1.723450,0.552274,8.087463,0.524782
ABC,-0.371142,-0.032169,0.524782,4.239379


In [257]:
df_ = pd.DataFrame()
df_['weights'] = [2,3,4,5]

In [258]:
df_

,weights
0,2
1,3
2,4
3,5


In [260]:
np.dot(cov_mat, df_.T)

ValueError: shapes (4,4) and (1,4) not aligned: 4 (dim 1) != 1 (dim 0)

In [293]:
df_stocks = pd.read_excel(r'./data/sp_12_weeks.xlsx')

In [299]:
portfolio = ['A', 'AAL', 'AAP', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'XEL', 'XLNX', 'XOM', 'XRAY', 'XRX', 'XYL', 'YUM', 'ZBH', 'ZION', 'XEL', 'XLNX', 'XOM', 'XRAY', 'XYL', 'YUM', 'ZBH', 'ZION',]

In [304]:
count = Counter(portfolio).items()
percentages = {x: float(float(y) / len(portfolio) * 100.00) for x, y in count}
            
list_of_stocks = percentages.keys()
weight_of_stocks = np.array([float(x) for x in list(percentages.values())])



In [305]:
cov_mat = df_stocks[list_of_stocks].cov()
pfolio_risk = np.sqrt(np.dot(weight_of_stocks.T, np.dot(cov_mat, weight_of_stocks)))

print(pfolio_risk)

55.13788866706454


In [302]:
print(weight_of_stocks)

[3.7037037  3.7037037  3.7037037  3.7037037  3.7037037  3.7037037
 3.7037037  3.7037037  3.7037037  3.7037037  7.40740741 7.40740741
 7.40740741 7.40740741 3.7037037  7.40740741 7.40740741 7.40740741
 7.40740741]


In [303]:
len(weight_of_stocks)

19

In [288]:
weight_of_stocks

array([3.7037037 , 3.7037037 , 3.7037037 , 3.7037037 , 3.7037037 ,
       3.7037037 , 3.7037037 , 3.7037037 , 3.7037037 , 3.7037037 ,
       7.40740741, 7.40740741, 7.40740741, 7.40740741, 3.7037037 ,
       7.40740741, 7.40740741, 7.40740741, 7.40740741])